<a href="https://colab.research.google.com/github/KennMujar/CS-Elec-2-Data-Mining/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PIP**

##### **PIP UNINSTALL**

In [ ]:
!pip uninstall -y transformers huggingface_hub
!pip uninstall -y spacy thinc

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: huggingface-hub 0.30.2
Uninstalling huggingface-hub-0.30.2:
  Successfully uninstalled huggingface-hub-0.30.2


##### **PIP INSTALL**

In [ ]:
!pip install -U transformers
!pip install datasets scikit-learn torch fsspec==2025.3.2

In [ ]:
!pip install datasets

In [ ]:
!pip install numpy==1.26.4 --quiet

# **GOOGLE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
import pandas as pd

# Load dataset
file_path = '/content/drive/MyDrive/DRegulars/SuicideDepression.csv'
df = pd.read_csv(file_path)
df = df[['text', 'label']].dropna() # drop empty rows

# Preview the dataset
df.head()

,text,label
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,"I’m so lostHello, my name is Adam (16) and I’v...",1


# **TOKENIZER**

In [ ]:
from transformers import BertTokenizer
from datasets import Dataset

# Using pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/243146 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset (80% train, 20% test)
split_dataset = dataset.train_test_split(test_size=0.2)

# Access the train and test sets
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

##### **Loading Model - Original**

In [ ]:
# Used for original code (not continuing epochs)
# Run this when running the original code

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##### **Loading Model - Epoch Continuation**

In [ ]:
# BUGS EVERYWHERE
# Used for continuing epochs
# Run this when continuing to run epochs (e.g. checkpoint-24316)

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/DRegulars/bert-suicidal-detector/checkpoint-24316", local_files_only=True
)

In [ ]:
import transformers
print(transformers.__version__)

4.51.3


# **TRAINING**

##### **Training Cell**

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/DRegulars/BERT - Checkpoints/bert-suicidal-detector',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=3,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

##### **Training Cell - Caller**

###### **Training Cell - Caller | Original**

In [ ]:
# Used for original code (not continuing epochs)
# Run this when running the original code
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rpp2022-7135-98184 (rpp2022-7135-98184-bicol-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


###### **Training Cell - Caller | Epoch Continuation**

In [ ]:
# Used for continuing epochs
# Run this when continuing to run epochs (e.g. checkpoint-24316)
trainer.train(resume_from_checkpoint=True)

# **CONTINUE TRAINING MODEL**

**Approach:**
1. Run "Training Cell" cell.
2. Do not run "Training Cell - Caller" cell.
3. In the cell below, change the checkpoint path found in "BERT - Checkpoints" directory.
4. Run the cell below.
5. NOTE: Only run one epoch, then continue and so on.


In [ ]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = BertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/DRegulars/bert-suicidal-detector/checkpoint-24316" # always change this path (found in drive "BERT - Checkpoints")
)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/DRegulars/BERT - Checkpoints/bert-suicidal-detector',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=3,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train(resume_from_checkpoint=True)

# **SAVING MODEL**

**NOTE:**
- Only will this cell below be run after all epochs is complete.

In [ ]:
save_directory = '/content/drive/MyDrive/DRegulars/Trained Models - BERT/bert-suicidal-detector'

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/DRegulars/Trained Models - BERT/bert-suicidal-detector')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/DRegulars/Trained Models - BERT/bert-suicidal-detector')